In [1]:
import pandas as pd
import numpy as np

In [6]:
#  id запроса и текст запроса, разделённые табуляцией
qid_query = pd.read_csv("/Users/dmitry/Downloads/yandex_cup_analytics_A/hidden_task/qid_query.tsv", 
                        sep="\t", 
                        names=["qid", "query"])
#  id запроса, URL документа, релевантность документа запросу;
qid_url_rating = pd.read_csv("/Users/dmitry/Downloads/yandex_cup_analytics_A/hidden_task/qid_url_rating.tsv", 
                             sep="\t", 
                             names=["qid", "url", "rating"])
# hostid_url.tsv — id хоста и URL документа.
hostid_url = pd.read_csv("/Users/dmitry/Downloads/yandex_cup_analytics_A/hidden_task/hostid_url.tsv", 
                         sep="\t", 
                         names=["hostid", "url"])

In [4]:
qid_query[:3]

,qid,query
0,402111,работа фотографом в австралии
1,405851,производительность видеокарт
2,407522,ёлочные игрушки из пластиковых бутылок


In [7]:
qid_url_rating[:3]

,qid,url,rating
0,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
1,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
2,402111,http://802351.info/5964-v-avstralii.html,0.00


In [8]:
hostid_url[:3]

,hostid,url
0,1,http://09spravki.ru/requisites.php
1,10,http://3pu.info/seo-tools/domains
2,1006,http://www.priroda.su/item/820


In [9]:
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [10]:
qid_url_rating_hostid[:5]

,qid,url,rating,hostid
0,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07,7
1,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07,7
2,402111,http://802351.info/5964-v-avstralii.html,0.00,13
3,402111,http://auscommunity.com/blog/jobs/,0.00,53
4,402111,http://auscommunity.com/tag/%D1%84%D0%BE%D1%82...,0.00,53


# PFound

In [69]:
def p_found(data: pd.DataFrame, 
            group_col: list, 
            rating_col: str, 
            K: int, 
            p_break: float = 0.15) ->float:
    
    # create data with max rating of each group per item
    max_by_group = data.groupby(
        group_col)[rating_col].max().reset_index().sort_values(
        [group_col[0], rating_col], ascending=False).groupby(group_col[0]).head(K)
    
    # create numbers(index) for each items
    max_by_group['numbers'] = max_by_group.groupby(group_col[0]).cumcount()
    
    # calculate (1 - Rel) and (1 - pBreak)
    max_by_group['1 - Rel'] = (1 - max_by_group[rating_col]).shift(1)
    max_by_group['1 - pBreak'] = 1 - p_break
    
    # first values for n == 0 will be 1
    max_by_group.loc[max_by_group['numbers'] == 0, ['1 - Rel', '1 - pBreak']] = 1
    
    # lets multiply (1 - Rel) and (1 - pBreak)
    max_by_group['multiply'] = max_by_group['1 - Rel'] * max_by_group['1 - pBreak']
    
    # Plook is a recursion function. For it we will use a pd.cumprod method
    max_by_group['Plook'] = max_by_group.groupby(group_col[0])['multiply'].cumprod()
    max_by_group['PFound'] = max_by_group['Plook'] * max_by_group[rating_col]
    
    return max_by_group.groupby(group_col[0])['PFound'].sum().mean()

In [71]:
p_found(qid_url_rating_hostid, ['qid', 'hostid'], 'rating', 5)

0.5370314635332146

# MRR

In [72]:
def generate_subsample(users_count, top_k):
    users = np.random.choice(interactions[Columns.User].unique(), users_count, replace=False)
    df = interactions[interactions[Columns.User].isin(users)].reset_index(drop=True)
    del df[Columns.Datetime], df[Columns.Weight], df['watched_pct']
    
    recs = np.random.choice(df[Columns.Item], size=(users_count, top_k))
    return df, users, recs